##### developing end-to-end model, where given an input and choosing model which is either a CNN or CANINE transfomer, it runs the model and returns the output of model for given input

In [89]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
import pickle

In [90]:
import torch
import pickle
from transformers import CanineTokenizer, CanineForSequenceClassification

In [91]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [92]:
# get the input from user
# text_input = input("Enter the input text: ")
model_input = input("Enter the model name, either CNN or CANINE: ")
dataset_input = input("Enter the dataset name, either yelp, dbpedia, or amazon: ")

if model_input == "CNN":
    # run CNN model
    model_flag = 0
else:
    # run CANINE model
    model_flag = 1

if dataset_input == "yelp":
    # run yelp dataset
    dataset_flag = 0
    text_input = input("Enter the input text: ")
elif dataset_input == "dbpedia":
    # run dbpedia dataset
    dataset_flag = 1
    title_input = input("Enter the title: ")
    content_input = input("Enter the content: ")
else:
    # run amazon dataset
    dataset_flag = 2
    title_input = input("Enter the title: ")
    content_input = input("Enter the content: ")


In [93]:
from datasets import load_dataset
if model_flag == 0:
    if dataset_flag == 0:
        dataset = load_dataset("yelp_review_full")
        train_dataset = dataset['train']
        test_dataset = dataset['test']

        # Convert train and test datasets to arrays
        train_data = train_dataset['text']
        train_labels = train_dataset['label']
        test_data = test_dataset['text']
        test_labels = test_dataset['label']
        
        # train val split
        train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)

    elif dataset_flag == 1:
        dataset = load_dataset("dbpedia_14")
        train_dataset = dataset['train']
        test_dataset = dataset['test']

        # Convert train and test datasets to arrays
        train_data_title = train_dataset['title']
        train_data_content = train_dataset['content']
        train_labels = train_dataset['label']
        test_data_title = test_dataset['title']
        test_data_content = test_dataset['content']
        test_labels = test_dataset['label']

        # train_data = train_data_title + train_data_content
        # test_data = test_data_title + test_data_content

        train_data = [None] * len(train_data_title)
        for i in range(len(train_data_title)):
            train_data[i] = train_data_title[i] + "mid" + train_data_content[i]

        test_data = [None] * len(test_data_title)
        for i in range(len(test_data_title)):
            test_data[i] = test_data_title[i] + "mid" + test_data_content[i]

        # train val split
        train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)

    else:
        dataset = load_dataset("amazon_polarity")
        train_dataset = dataset['train']
        test_dataset = dataset['test']

        # Convert train and test datasets to arrays
        train_data_title = train_dataset['title']
        train_data_content = train_dataset['content']
        train_labels = train_dataset['label']
        test_data_title = test_dataset['title']
        test_data_content = test_dataset['content']
        test_labels = test_dataset['label']

        # train_data = train_data_title + train_data_content
        # test_data = test_data_title + test_data_content

        train_data = [None] * len(train_data_title)
        for i in range(len(train_data_title)):
            train_data[i] = train_data_title[i] + "mid" + train_data_content[i]

        test_data = [None] * len(test_data_title)
        for i in range(len(test_data_title)):
            test_data[i] = test_data_title[i] + "mid" + test_data_content[i]

        # train val split
        train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)



In [94]:
if dataset_flag == 1:
    # define text_input as title_input + mid + content_input
    text_input = title_input + "mid " + content_input
if dataset_flag == 2:
    # define text_input as title_input + mid + content_input
    text_input = title_input + "mid " + content_input

In [95]:
# pre-process and run inference the input text based on which dataset is chosen and which model is chosen
if model_flag == 1:
    tokenizer = CanineTokenizer.from_pretrained("google/canine-c")
    encoded_input = tokenizer(text_input, padding=True, truncation=True,return_tensors="pt").to(device)

    if dataset_flag == 0:
        # load the yelp model
        loaded_model = CanineForSequenceClassification.from_pretrained("canine_yelp_model_2").to(device)    
    elif dataset_flag == 1:
        # load the dbpedia model
        loaded_model = CanineForSequenceClassification.from_pretrained("canine_dbpedia_model_2").to(device)
    else:
        # load the amazon model
        loaded_model = CanineForSequenceClassification.from_pretrained("canine_amazon_model_2").to(device)

    #perform inference
    with torch.no_grad():
        loaded_model.eval()
        output = loaded_model(**encoded_input)
        # scores = output[0][0].cpu().numpy()
        # print("The output scores are: ", scores)
        predicted_label = torch.argmax(output.logits, dim=1).item()

    # print the predicted label
    print("The predicted label is: ", predicted_label)

else:
    tokenizer_cnn  = Tokenizer(char_level=True)
    tokenizer_cnn.fit_on_texts(train_data)
    sequences = tokenizer_cnn.texts_to_sequences([text_input])
    # padded_sequences = pad_sequences(sequences, maxlen=1495)


    if dataset_flag == 0:
        # load the yelp model
        padded_sequences = pad_sequences(sequences, maxlen=1100)
        loaded_model = keras.models.load_model('yelp_cnn_model.h5')
    elif dataset_flag == 1:
        # load the dbpedia model
        # print("The sequence is: ", sequences)
        padded_sequences = pad_sequences(sequences, maxlen=1495)
        # print("The padded sequence is: ", padded_sequences)
        loaded_model = keras.models.load_model('dbpedia_cnn.h5')
    else:
        # load the amazon model
        padded_sequences = pad_sequences(sequences, maxlen=266)
        loaded_model = keras.models.load_model('amazon_cnn.h5')

    #perform inference
    output = loaded_model.predict(padded_sequences)
    predicted_label = np.argmax(output, axis=1)

    # print the predicted label
    print("The predicted label is: ", predicted_label)

1/1 [==============================] - 0s 43ms/step
The predicted label is:  [1]


# example cases:

- dbpeida:
    - input titel: TY KU
    - input content: TY KU /taɪkuː/ is an American alcoholic beverage company that specializes in sake and other spirits. The privately-held company was founded in 2004 and is headquartered in New York City New York. While based in New York TY KU's beverages are made in Japan through a joint venture with two sake breweries. Since 2011 TY KU's growth has extended its products into all 50 states.
    - output class: 0

    - input title: Weybridge Man Powered Aircraft
    - input content: The Weybridge Man Powered Aircraft (also known as Dumbo and later Mercury) is a British single-seat man-powered aircraft built and flown by members of the Weybridge Man Powered Aircraft Group.
    - output class: 5

- yelp:
    - input: This cafe's espresso drinks are on par with the best artisinal coffee houses anywhere. I always go out of my way to get coffee from here when I'm in Pittsburgh.
    - output class: 4

    - input: Maybe I am just the most jaded vegan in town; Quiet Storm's food menu bores me to tears, that which I will use to salt my own mediocre cooking at home! Their coffee isn't all that great either, as given the choice, local La Prima is in my favor. Tears. This is the only vegetarian restaurant on the East side of town, and oh so close to me, but I cannot stand to go there. Desperate times now call for toast and tea served by the sad server that resides within my depressed little dining room.
    - output class: 0

- amazon:
    - input title: Hunting the Hard Way
    - input content: Thia was a gift for my Husband, who loved the book. It arrived on the date we were told it would.
    - output class: 1

    - input title: Disappointing
    - input content: The text is the same as ever (I think), but the plates are markedly inferior to all previous editions. Had I known this ahead of time, I would not have purchased this volume.
    - output class: 0